# Computing deltas with a local dask cluster

Compute deltas for a single model and scenario with a local Dask cluster.

In [1]:
from pathlib import Path
from dask.distributed import Client
import pandas as pd
import xarray as xr

Start the client (creates a local cluster)

In [2]:
client = Client(dashboard_address="localhost:43368", n_workers=16)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43368/status,
Dashboard: http://127.0.0.1:43368/status,Workers: 16
Total threads: 32,Total memory: 251.71 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33512,Workers: 16
Dashboard: http://127.0.0.1:43368/status,Total threads: 32
Started: Just now,Total memory: 251.71 GiB
Comm: tcp://127.0.0.1:35435,Total threads: 2
Dashboard: http://127.0.0.1:38469/status,Memory: 15.73 GiB
Nanny: tcp://127.0.0.1:33344,


In [3]:
%%time
data_dir = "/atlas_scratch/kmredilla/AK_NCAR_12km/met/CCSM4/rcp45/"
paths = [Path(data_dir).joinpath(f"CCSM4_rcp45_BCSD_met_{year}.nc4") for year in range(1980, 2100)]
ds = xr.open_mfdataset(paths)

CPU times: user 15.8 s, sys: 2.57 s, total: 18.3 s
Wall time: 29.4 s


Aggregate by year. Could expand to create other data variables for e.g. indicators.

In [17]:
# performing different aggregate operations for each 
year_ds = xr.merge([
    ds["pcp"].groupby("time.year").sum(min_count=1e-9),
    ds["tmax"].groupby("time.year").max(skipna=False),
    ds["tmin"].groupby("time.year").min(skipna=False)
])

Climatology operation. Assign an "era" coordinate to the dataset (applies to each climate variable DataArray within). This will be used so we can groupby era and compute climatologies.

In [20]:
# assign an era coordinate for grouping
# using pandas.cut function for binning years into eras
clim_ds = year_ds.assign_coords(
    era=(
        "year", 
        pd.cut(
            year_ds.year,
            [1980, 2010, 2040, 2070, 2100],
            labels=["hist", "near", "mid", "far"]
        )
    )
).groupby("era").mean()

Deltas operation:

In [21]:
# using concat because we need to tell xarray what to call
#  era for each delta (just the future era being used)
future_eras = ["near", "mid", "far"]
deltas_ds = xr.concat([
    xr.merge([
        clim_ds[varname].sel(era=era) - clim_ds[varname].sel(era="hist")
        for varname in clim_ds.data_vars
    ])
    for era in future_eras
], pd.Index(future_eras, name="era"))

Run the computation:

In [22]:
%%time
deltas_ds = deltas_ds.compute()

CPU times: user 14.8 s, sys: 1.74 s, total: 16.5 s
Wall time: 20.1 s


Visualize the results.

In [77]:
import hvplot.xarray
import panel.widgets as pnw
import panel as pn
# local script to help reproject
import reproject_ncar


reproj_ds = reproject_ncar.reproject_ncar(deltas_ds).drop("spatial_ref")
variable_select = pnw.Select(options={v: v for v in reproj_ds.data_vars})

eras_rbg = pn.widgets.RadioButtonGroup(
    name="Display", options={str(era): era for era in reproj_ds.era.values}, button_type="default"
)
cmap_rbg = pn.widgets.RadioButtonGroup(options={"BrBG": "BrBG", "RdBu_r": "RdBu_r"})

plot_da = reproj_ds.interactive()[variable_select].sel(era=eras_rbg)

@pn.depends(varname=variable_select)
def get_clim(varname):
    da = reproj_ds[varname]
    return (0 - da.max(), da.max())

@pn.depends(varname=variable_select)
def get_cmap(varname):
    cmap_di = {"pcp": "BrBG", "tmax": "RdBu_r", "tmin": "RdBu_r"}
    return cmap_di[varname]

plot_da.hvplot(cmap=get_cmap, kind="image", clim=get_clim).opts(width=800, height=500)